# Install packages and load data

In [ ]:
!pip install folium --quiet
!pip install geoviews --quiet
!pip install Cartopy  --quiet
!pip install geopandas geoviews --quiet
!pip install hvplot --quiet
!pip install --upgrade hvplot param  --quiet
!pip install plotly pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.2/511.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.1 MB/s eta 0:00:00


In [ ]:
import folium
import pandas as pd
import panel as pn
import param
from folium import plugins
from folium.plugins import HeatMap
import numpy as np
pn.extension()

#####################################
# connecting with Google Drive
import gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default

creds, _ = default()
gcolab = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 #####################################
# load data
gsheets = gcolab.open_by_url('https://docs.google.com/spreadsheets/d/1mer2kkjM7u-jJGsN0OXr4pne84SYk/')
sheets = gsheets.sheet1.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0]).set_index("doc_id")

In [ ]:
# select and type data
df = df[["score_relevance", "title", "doc_date", "news_mentions", "city", "country_code", "latitude", "longitude"]]
df['doc_date'] = pd.to_datetime(df['doc_date'])
df['latitude'] = df['latitude'].astype('float')
df['longitude'] = df['longitude'].astype('float')
df['score_relevance'] = df['score_relevance'].astype('float')
df = df[df['score_relevance'] > 0.1]
df=df[df["doc_date"] >= "2019-01-01"]

In [ ]:
df.head()

,score_relevance,title,doc_date,news_mentions,city,country_code,latitude,longitude
doc_id,,,,,,,,
1286105761,0.8854,Aberdeenshire university graduate celebrates S...,2023-10-25,[],Aberdeen,GB,57.14369,-2.09814
672181558,0.7616,Drivers need more training to cope with semi-a...,2020-11-04,[],Aberdeen,GB,57.14369,-2.09814
672036942,0.7616,Trial of driverless car technology launched in...,2020-10-30,[],Aberdeen,GB,57.14369,-2.09814
668408055,0.7616,New research project looks to unlock urban aut...,2020-10-12,[],Aberdeen,GB,57.14369,-2.09814
662921741,0.7616,Researchers claim to develop concept to ensure...,2020-09-14,[],Aberdeen,GB,57.14369,-2.09814


# Define dashboard

In [ ]:
date_min = df['doc_date'].min()
date_max = df['doc_date'].max()

# compute max. and min. number of articles per city
df_agg = df.groupby('city').agg(nb_articles = ('city', np.size))
nmin = df_agg["nb_articles"].min()
nmax = df_agg["nb_articles"].max()
smin = 5.0
smax = 60.0
min_articles = 4
max_articles = 3

class NewsDashboard(param.Parameterized):
    date_range = param.DateRange(default=(date_min, date_max), bounds=(date_min, date_max))

    def _get_filtered_data(self):
        df1 = df.copy()  # Make a copy to avoid modifying the original dataframe

        # Convert start_date and end_date to pandas datetime64[ns] objects
        start_date, end_date = map(pd.to_datetime, self.date_range)

        # Filter data based on the selected date range
        df1 = df1[
            (df1['doc_date'] >= start_date) & (df1['doc_date'] <= end_date)
        ]
        df1 = df1.sort_values("doc_date", ascending = False)
        df1["article"] = "<i>" + df1["doc_date"].astype(str) + "</i>: " + df1["title"]

        df1 = df1.groupby('city').agg(nb_articles = ('city', np.size), articles = ('article', list), dates = ('doc_date', list), latitude = ('latitude', np.mean), longitude = ('longitude', np.mean)).reset_index()
        df1 = df1[df1["nb_articles"] >= min_articles]

        df1["bubble_size"] = (df1["nb_articles"]**2 - nmin**2) / (nmax**2 - nmin**2) * smax + smin
        #df1["tooltip"] = "# articles: " + df1["nb_articles"].astype("str") + " - " + df1["city"]
        df1["popup"] = ["<br>".join(x[0:max_articles]) + "..." for x in df1["articles"]]
        df1["popup"] = ["<b>" + row["city"] + "</b> - " + str(row["nb_articles"]) + " articles:<br><br>" + row["popup"] for _, row in df1.iterrows()]

        df1 = df1.set_index("city")
        df1 = df1.sort_values("nb_articles", ascending = False)

        return df1

    """
    #@param.depends("date_range")
    def view_table(self):
        df1 = self._get_filtered_data()
        df1 = df1[["nb_articles"]]

        return pn.widgets.DataFrame(df1, name="News articles", width = 400, height=600, widths={"index": 300, "nb_articles": 100})
    """

    #@param.depends("date_range")
    def view_map(self):
        df1 = self._get_filtered_data()
        m = folium.Map(location=[10, 10], zoom_start=2)
        for _, row in df1.iterrows():
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=row['bubble_size'],
                popup=folium.Popup(row["popup"], min_width=300, max_width=300),
                #tooltip=row["tooltip"],
                fill=True,
                fill_color='blue',
                stroke=False,
                fill_opacity=0.4,
            ).add_to(m)
        #HeatMap(df1[["latitude", "longitude"]]).add_to(m)
        return pn.pane.HTML(m._repr_html_(), width=1200, height=800)

    def view(self):
        return pn.Column(
            pn.Row(
                pn.Column(
                    pn.layout.Spacer(height=20),  # Spacer for vertical alignment
                    pn.Param(self.param.date_range, width=400),  # Reduced width for the date range slider
                ),
                align='start',  # Align the row content to the left
            ),
            pn.Row(
                pn.Column(
                    self.view_map,
                ),
                #pn.Column(
                    #self.view_table
                #)
            ),
            styles=dict(background='WhiteSmoke')
        )

# Run dashboard

In [ ]:
dashboard = NewsDashboard()
dashboard.view().servable()

Column(styles={'background': '...})
    [0] Row
        [0] Column
            [0] Spacer(height=20)
            [1] Param(NewsDashboard, show_name=False, width=400)
    [1] Row
        [0] Column
            [0] ParamMethod(method, _pane=HTML, defer_load=False)

/usr/local/lib/python3.10/dist-packages/param/parameterized.py:1720: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return eq(obj1, obj2)
/usr/local/lib/python3.10/dist-packages/param/parameterized.py:1720: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return eq(obj1, obj2)
